#  Forward pass

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann 
* https://pytorch.org/docs/stable/generated/torch.matmul.html
* https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/
* https://machinelearningmastery.com/loss-and-loss-functions-for-training-deep-learning-neural-networks/
* https://kidger.site/thoughts/jaxtyping/
* https://github.com/patrick-kidger/torchtyping/tree/master

## Задачи для совместного разбора

1\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте нейрон с заданными весами `weights` и `bias`. Пропустите вектор `inputs` через нейрон и выведите результат. 

In [3]:
import torch as th

class Neuron:
    def __init__(self, n_features: int, bias: bool = True):
        # <создать атрибуты объекта weights и bias>
        self.weights: th.Tensor = th.randn(n_features)
        if bias:
            self.bias: float = th.randn(1)
        else:
            self.bias = 0

    def forward(self, inputs: th.Tensor) -> th.Tensor:
        # inputs: (n_features, )
        y = self.weights @ inputs + self.bias
        
        # returns scalar
        return y


In [4]:
import torch as th
inputs = th.tensor([1.0, 2.0, 3.0, 4.0])

neuron = Neuron(
    n_features=len(inputs),
    bias = True
)

neuron.forward(inputs)

tensor([6.9957])

2\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации.

In [5]:
class ReLU:
    def forward(self, inputs: th.Tensor) -> th.Tensor:
        # <реализовать логику ReLU>
        # inputs: (n_features, )
        # returns: (n_features, )
        return inputs.clip(min=0)

In [6]:
inputs = th.randn(4,3)
act = ReLU()
act.forward(inputs)

tensor([[0.7687, 0.4377, 1.0318],
        [0.0000, 0.6162, 0.0000],
        [0.0000, 0.0342, 0.0341],
        [0.0000, 1.1235, 0.0000]])

3\. Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)
где $Y_i$ - правильный ответ для примера $i$, $\hat{Y_i}$ - предсказание модели для примера $i$, $n$ - количество примеров в батче.

In [7]:
class MSELoss:
    def forward(self, y_pred: th.Tensor, y_true: th.Tensor) -> th.Tensor:
        # y_pred: (batch_size, )
        # y_true: (batch_size, )
        # returns: scalar
        return (y_true - y_pred).pow(2).mean()

In [8]:
y_pred = th.tensor([1.0, 3.0, 5.0])
y_true = th.tensor([2.0, 3.0, 4.0])

MSELoss().forward(y_pred,y_true)

tensor(0.6667)

## Задачи для самостоятельного решения

### Cоздание полносвязных слоев

<p class="task" id="1"></p>

1\. Используя операции над тензорами из библиотеки `torch`, реализуйте полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения) и опциональным вектором смещения. Для определения слоя не используйте готовые функции или слои из `torch`, реализуйте слой, используя только операции над тензорами.

$$y = xW^T + b$$

Пропустите вектор `inputs` через слой и выведите результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

- [x] Проверено на семинаре

In [9]:
import torch as th
class Linear:
    def __init__(self, n_neurons: int, n_features: int, bias: bool = False) -> None:
        self.n_features = n_features
        self.n_neurons = n_neurons

        self.weights: th.Tensor = th.randn(n_neurons,n_features)
        if bias:
            self.bias: float = th.randn(1)
        else:
            self.bias = 0

    def forward(self, inputs: th.Tensor) -> th.Tensor:
        # inputs: (batch_size, n_features)
        # returns: (batch_size, n_neurons)
        return inputs @  self.weights.T + self.bias

inputs = th.randn(5,6)
n_neurons = 7
print(f'batch_size x n_features = {inputs.shape}')

nthlin = Linear(n_neurons,inputs.shape[1],True)

res = nthlin.forward(inputs)
print(f'batch_size x n_neurons = {res.shape}')

batch_size x n_features = torch.Size([5, 6])
batch_size x n_neurons = torch.Size([5, 7])


<p class="task" id="2"></p>

2\. Используя решение предыдущей задачи, создайте 2 полносвязных слоя и пропустите тензор `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выберите произвольно, количество нейронов во втором слое выберите так, чтобы результатом прогона являлась матрица `batch_size x 7`. 

- [ ] Проверено на семинаре

In [10]:
inputs = th.randn(50,60)
n_neurons1 = 20

print(f'batch_size x n_features = {inputs.shape}')


nthlin1 = Linear(n_neurons1,inputs.shape[1],True)
res1 = nthlin1.forward(inputs)

nthlin2 = Linear(7,res1.shape[1],True)
res2 = nthlin2.forward(res1)
print(f'batch_size x 7 = {res2.shape}')

batch_size x n_features = torch.Size([50, 60])
batch_size x 7 = torch.Size([50, 7])


### Создание функций активации

<p class="task" id="3"></p>

3\. Используя операции над тензорами из библиотеки `torch`, реализуйте функцию активации softmax:

$$f_i(\vec{x}) = \frac{e^{x_i}}{\sum_{j=1}^J e^{x_j}}$$

$$\overrightarrow{x} = (x_1, ..., x_J)$$

Создайте матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров. Функция должна применяться переданной на вход матрице построчно. Для определения слоя не используйте готовые функции или слои из `torch`, реализующие данную функцию активации.

- [ ] Проверено на семинаре

In [11]:
import torch as th
class Softmax:
    def forward(self, inputs: th.Tensor) -> th.Tensor:
        # <реализовать логику Softmax>
        # inputs: (batch_size, n_features)
        # returns: (batch_size, n_features)
        exp = th.exp(inputs)
        return exp / th.sum(exp,1, keepdim=True)
    
inputs = th.randn(4,3)
    
sm = Softmax()
sm.forward(inputs)

tensor([[0.6341, 0.1356, 0.2303],
        [0.2099, 0.4611, 0.3290],
        [0.6463, 0.1357, 0.2180],
        [0.2647, 0.2855, 0.4498]])

<p class="task" id="4"></p>

4 Используя операции над тензорами из библиотеки `torch`, реализуйте функцию активации ELU:

$$f(\alpha,x) = \begin{cases}\alpha(e^x - 1) \\ x\end{cases} ~ \begin{gather}x <0 \\ x \ge 0\end{gather}$$

Создайте матрицу размера 4x3, заполненную числами из стандартного нормального распределения, и проверьте работоспособность функции активации. Для определения слоя не используйте готовые функции или слои из `torch`, реализующие данную функцию активации.

- [ ] Проверено на семинаре

In [12]:
import torch as th
class ELU:
    def __init__(self, alpha: float) -> None:
        self.alpha = alpha
    
    def forward(self, inputs: th.Tensor) -> th.Tensor:
        
        # <реализовать логику ELU>
        # inputs: (batch_size, n_features)
        # returns: (batch_size, n_features)
        return th.where(inputs>=0,inputs,self.alpha * (th.exp(inputs) - 1))
    
inputs = th.randn(4,3)
    
elu = ELU(0.1)
elu.forward(inputs)

tensor([[-0.0730,  0.2091,  0.5829],
        [-0.0737, -0.0566,  0.2634],
        [ 1.0739,  0.0409,  2.1821],
        [ 0.7740, -0.0026, -0.0785]])

### Создание функции потерь

<p class="task" id="5"></p>

5 Используя операции над матрицами и векторами из библиотеки `torch`, реализуйте функцию потерь CrossEntropyLoss:



<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

$$ CrossEntropyLoss = \frac{1}{n}\sum_{i=1}^{n}{L_i}$$
где $y_i$ - вектор правильных ответов для примера $i$, $\hat{y_i}$ - вектор предсказаний модели для примера $i$; $k$ - количество классов, $n$ - количество примеров в батче. Для определения слоя не используйте готовые функции или слои из `torch`, реализующие данную функцию потерь.

Создайте полносвязный слой с 2 нейронами и прогнать через него батч `inputs`. Полученный результат пропустите через функцию активации Softmax. Посчитайте значение функции потерь, трактуя вектор `y` как вектор правильных ответов. 

- [ ] Проверено на семинаре

In [36]:
import torch as th
class CrossEntropyLoss:
    def forward(self, y_pred: th.Tensor, y_true: th.Tensor) -> th.Tensor:
        # <реализовать логику функции потерь>
        # y_pred: (batch_size, n_classes)
        # y_true: (batch_size, n_classes)
        # returns: scalar
        return (-1/ y_true.shape[0]) * (y_true * th.log(y_pred)).sum()


inputs = th.randn(7,5)

lin = Linear(2,inputs.shape[1], True)
res = lin.forward(inputs)

act = Softmax()
res = act.forward(res)

ce = CrossEntropyLoss()
loss = ce.forward(res[:,0].reshape(-1,1),res[:,1].reshape(-1,1))
loss

tensor(0.7596)

<p class="task" id="6"></p>

6 Модифицируйте MSE, добавив L2-регуляризацию.

$$MSE_R = MSE + \lambda\sum_{i=1}^{m}w_i^2$$

где $\lambda$ - коэффициент регуляризации; $w_i$ - веса модели.

- [ ] Проверено на семинаре

In [40]:
import torch as th
class MSERegularized:
    def __init__(self, lambda_: float) -> None:
        self.lambda_ = lambda_

    def data_loss(
        self, 
        y_pred: th.Tensor, 
        y_true: th.Tensor,
    ) -> th.Tensor:
        # <подсчет первого слагаемого из формулы>
        # y_pred: (batch_size, )
        # y_true: (batch_size, )
        return (y_true - y_pred).pow(2).mean()

    def reg_loss(self, weights: th.Tensor) -> th.Tensor:
        # <подсчет второго слагаемого из формулы>
        # weights: (batch_size, 1)
        return self.lambda_ * weights.pow(2).sum()

    def forward(
        self, 
        y_pred: th.Tensor, 
        y_true: th.Tensor, 
        weights: th.Tensor,
    ) -> th.Tensor:
        # y_pred: (batch_size,)
        # y_true: (batch_size,)
        # weights: (batch_size, 1)
        return self.data_loss(y_pred, y_true) + self.lambda_ * self.reg_loss(weights)
    

y_pred = th.tensor([1.0, 3.0, 5.0])
y_true = th.tensor([2.0, 3.0, 4.0])

MSERegularized(0.2).forward(y_pred,y_true, th.tensor([0.1,0.2,0.3]))

tensor(0.6723)